# 使用新方法计算F1
- F1计算方式为表格中的信息被模型抽取到

In [1]:
# file_path = 'results/baichuan2-13B-iepile-data2text-test-output.json'
file_path = 'results/baichuan7B-ft-test-output-v2.json'
# file_path = 'results/baichuan2-13B-iepile-ft-test-output.json'

In [73]:
import json

excluded_entity_types = {"任务", "事件", "行动"}
# excluded_entity_types = {'任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '行动', '军种', '数量', '任务状态', '开始时间', '装备', '单位', '人数', '国别',}


def count_entities_in_jsonl(file_path):
    len_label = 0

    # 打开 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # 逐行读取文件内容
        for line in file:
            # 解析 JSON 数据
            record = json.loads(line)

            # 获取 label 字段并解析为列表
            label = json.loads(record['label'])

            for la in label:
                if la['entity_type'] in excluded_entity_types:
                    continue
                len_label += 1
                
    # “6”表示headers总数22（25种除去行动事件任务）/split_num（一次抽取4个实体类型）后向上取整
    return len_label / 6


# 设置你的文件路径
total_entities = count_entities_in_jsonl(file_path)
print(f"总实体数量: {total_entities}")


总实体数量: 1420.0


In [74]:
import json

excluded_entity_types = {"任务", "事件", "行动"}
# excluded_entity_types = {'组织机构', '任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '行动', '军种', '数量', '任务状态', '时间', '地点', '人物', '开始时间', '装备名称', '装备', '单位', '人数', '装备类型', '事件', '国家', '任务', '国别', '装备数量'}

def count_valid_entities(file_path):
    valid_entity_count = 0

    # 打开 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # 逐行读取文件内容
        for line in file:
            # 解析 JSON 数据
            record = json.loads(line)

            # 解析 label 和 output 字段
            label_entities = json.loads(record['label'])
            output_entities = json.loads(record['output'])

            # 将 output 中的实体合并成一个列表
            output_entity_list = []
            for key in output_entities:

                output_entity_list.extend(output_entities[key])

            # print(output_entity_list)

            # 检查 label 中的每个实体是否在 output 中逐字匹配
            for entity_info in label_entities:
                label_entity = entity_info['entity']
                entity_type = entity_info['entity_type']

                if entity_type in excluded_entity_types:
                    continue

                # 检查是否 label_entity 在 output_entity_list 中的任何一个实体字符串中
                if any(label_entity in output_entity for output_entity in output_entity_list):
                    valid_entity_count += 1
                # else:
                #     print(label_entity)

    return valid_entity_count


# 设置你的文件路径
valid_entity_count = count_valid_entities(file_path)
print(f"有效实体数量: {valid_entity_count}")

# 
print(f"F1值：{valid_entity_count / total_entities}")

有效实体数量: 1239
F1值：0.8725352112676056


{
  '组织机构':0.769
  '任务类型': 
  '备注':
  '指挥员': 
  '主体数量':0.819 
  '主体名称':0.851
  '主体类型':0.811
  '军种': 
  '数量': 
  '任务状态': 
  '时间': 0.901
  '地点': 0.913
  '人物': 0.08 # 原始数据标注质量不好
  '开始时间': 
  '装备名称':0.866 
  '装备':
  '单位': 
  '人数': 
  '装备类型': 0.841
  '国家': 0.365 
  '国别':
  '装备数量':0.817
}

## 计算F1

In [2]:
import json

# excluded_entity_types = {"装备数量"}
excluded_entity_types = {'组织机构', '任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '军种', '数量', '任务状态', '时间', '地点', '开始时间', '装备名称', '装备', '单位', '人数', '装备类型', '国别', '装备数量', '人物', '国家'}
# excluded_entity_types = {'组织机构', '任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '军种', '数量', '任务状态', '时间', '地点', '开始时间', '装备名称', '装备', '单位', '人数', '装备类型', '国别', '装备数量'}


def count_valid_entities(file_path):
    valid_entity_count = 0
    entity_count = 0

    # 打开 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # 逐行读取文件内容
        for line in file:
            # 解析 JSON 数据
            record = json.loads(line)

            # 解析 label 和 output 字段
            label_entities = json.loads(record['label'])
            output_entities = json.loads(record['output'])

            # 将 output 中的实体合并成一个列表
            output_entity_list = []
            for key in output_entities:
                output_entity_list.extend(output_entities[key])

            # print(output_entity_list)

            # 检查 label 中的每个实体是否在 output 中逐字匹配
            for entity_info in label_entities:
                label_entity = entity_info['entity']
                entity_type = entity_info['entity_type']

                if entity_type in excluded_entity_types:
                    entity_count += 1
                    for key in output_entities:
                        # 检查是否 label_entity 在 output_entity_list 中的任何一个实体字符串中
                        if key == entity_type:
                            if any(label_entity in output_entity for output_entity in output_entity_list):
                                valid_entity_count += 1

    return valid_entity_count, entity_count / 6


# 设置你的文件路径
valid_entity_count, entities = count_valid_entities(file_path)
print(f"有效实体数量: {valid_entity_count}，总实体数量：{entities}")

# 算F1
print(f"F1值：{valid_entity_count / entities}")

有效实体数量: 978，总实体数量：1420.0
F1值：0.6887323943661972


## 统计
{
  '组织机构':0.769
  '任务类型': 
  '备注':
  '指挥员': 
  '主体数量':0.819 
  '主体名称':0.851
  '主体类型':0.811
  '军种': 
  '数量': 
  '任务状态': 
  '时间': 0.901
  '地点': 0.913
  '人物': 0.08 # 原始数据标注质量不好
  '开始时间': 
  '装备名称':0.866 
  '装备':
  '单位': 
  '人数': 
  '装备类型': 0.841
  '国家': 0.365 
  '国别':
  '装备数量':0.817
}